In [ ]:
!rm -r /content/outputs

rm: cannot remove '/content/outputs': No such file or directory


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

import os
os.environ["WANDB_DISABLED"] = "true"
print(os.environ["WANDB_DISABLED"])

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="auto",
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!nvidia-smi

Sat Jun  1 13:19:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0              28W /  70W |   5671MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
finetuning_template = """Below is a text in markdown with all information in french about the faculty of science and techniques in Tanger. The text should expand your knowledge about the faculty of science and techniques in Tanger but not give you an exact way to respond.

**Text**
{}"""

prompt_template = """Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    responses      = examples["response"]
    texts = []
    for instruction, response in zip(instructions, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_template.format(instruction, response) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def formatting_finetuning_func(examples):
    texts = examples["text"]
    formated_texts = []
    for txt in texts:
        formated_text = finetuning_template.format(txt) + EOS_TOKEN
        formated_texts.append(formated_text)
    return { "text" : formated_texts, }
pass

from datasets import load_dataset, concatenate_datasets
text_dataset = load_dataset("json", data_files="/content/sample_data/uni_info_text.json", split = "train")
text_dataset = text_dataset.map(formatting_finetuning_func, batched = True,)

instructions_dataset = load_dataset("json", data_files="/content/sample_data/instructions.json", split = "train")
instructions_dataset = instructions_dataset.map(formatting_prompts_func, batched = True,)

instructions_dataset = instructions_dataset.remove_columns(['instruction', 'response'])

dataset = concatenate_datasets([text_dataset, instructions_dataset])

dataset

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1500,
        num_train_epochs = 1500,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to=None,
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/2274 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.762 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,274 | Num Epochs = 6
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,500
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,2.967700
2,3.066200
3,2.976000
4,2.770500
5,2.511900
6,2.250000
7,2.027800
8,1.855200
9,1.931800
10,1.444600


Step,Training Loss
1,2.967700
2,3.066200
3,2.976000
4,2.770500
5,2.511900
6,2.250000
7,2.027800
8,1.855200
9,1.931800
10,1.444600


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

11943.1475 seconds used for training.
199.05 minutes used for training.
Peak reserved memory = 9.998 GB.
Peak reserved memory for training = 4.236 GB.
Peak reserved memory % of max memory = 67.792 %.
Peak reserved memory for training % of max memory = 28.723 %.


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Qui dirige le département de Génie Mécanique?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Qui dirige le département de Génie Mécanique?

### Response:
ELAYACHI Ilham<|eot_id|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Presente moi la FST Tanger?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Presente moi la FST Tanger?

### Response:
La FST de Tanger est une institution publique relevant de l’enseign

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Donne moi toutes les methodes pour contacter la FST Tanger?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Donne moi toutes les methodes pour contacter la FST Tanger?

### Response:
Vous pouvez contacter la FST Tanger

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Qui est MUSTAPHA DIANI?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Qui est MUSTAPHA DIANI?

### Response:
Le Doyen de l'établissement<|eot_id|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Qui sont les représentants des P.E.S?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Qui sont les représentants des P.E.S?

### Response:
AOUNI Abdesamad, BOUASSAB Abderrahman, EL METOUI Mustapha

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Donne moi les représentants des P.E.S", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Donne moi les représentants des P.E.S

### Response:
AOUNI Abdesamad, BOUASSAB Abderrahman, EL METOUI Mustapha

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Quand est-ce que les contrôles continus N°2 (CC2) du semestre de Printemps 2016-2017 pour les cycles MASTER et INGENIEUR vont demarrer", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Quand est-ce que les contrôles continus N°2 (CC2) du semestre de Printemps 2016-2017 pour les cycles MASTER et

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Donne moi le programme du 1er semestre de de la formation master en intelligence artificielle et science de données", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that queries or asks about something regarding the faculty of science and techniques in Tanger. Always write a response in French that appropriately answers a question or respondes to an instruction. You should be always a helpful and respectful assistant. You can be creative but within the provided context, the context of the faculty of science and techniques in Tanger, and an educational context. The answer should be a meaningful sentence, not just a point blank response to the instruction, but it should be consice to the instruction, do not add anymore information. If the question or instruction does not concern the faculty of science and techniques in Tanger, or outside an educational context, or you do not dispose of an answer, responde with: "Cette information n'existe pas ou n'est pas disponible pour le public."

### Instruction:
Donne moi le programme du 1er semestre de de la formation master en intelligence artificielle et science de do

In [ ]:
model.push_to_hub("hf/unsloth-llama-3-lora", token = "hf_token") # Online saving
tokenizer.push_to_hub("hf/unsloth-llama-3-lora", token = "hf_token") # Online saving

In [ ]:
model.push_to_hub_gguf("hf/unsloth-llama-3-q4_k_m", tokenizer, quantization_method = "q4_k_m", token = "hf_token")

In [ ]:
model.push_to_hub_gguf("hf/unsloth-llama-3-f16", tokenizer, quantization_method = "f16", token = "hf_token")